In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os 
from pkg import detrend_group
os.chdir("/Users/caropark/FAO_ag_check_code")

data originally comes from combined_update_dt.pkl (on local disc)... then modified to .csv.
and now fixing again

redoing the avg_2crops_n_lags(df) function in 0B.csif_proxy_fx_update

In [15]:
df = pd.read_csv("./data/yield_comparison.csv")#.drop({"yield_dt", "yield_lag", "yield_lead", "whichlag", "yield"}, axis=1)
df = df.loc[:, ~df.columns.str.contains("_log|_lead|lag", regex=True)]

In [16]:
df = detrend_group(df, "yield_og", "yield_log_dt", log_transform=True)
df = detrend_group(df, "csif_og", "csif_log_dt", log_transform= True)
df['yield_lag'] = df.groupby(['cropname', 'country'])['yield_log_dt'].shift(1)
df['yield_lead'] = df.groupby(['cropname','country'])['yield_log_dt'].shift(-1)

In [52]:
corrs = (df.groupby(["cropname", "country"])
      .apply(lambda g: g[["yield_log_dt", "yield_lag", "yield_lead"]].corrwith(g["csif_log_dt"]) if len(g) >= 2 else None).dropna())
corrs = corrs.idxmax(axis=1).to_frame().reset_index().rename({0: 'whichlag'}, axis=1)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


In [55]:
df = df.merge(corrs)
df['name'] = np.where(df.whichlag == "yield_log_dt", df.country, df.country + " (" + (df.whichlag).str.replace("yield_", "") + ")")

In [56]:
df.to_csv("./data/yield_comparison.csv", index=False)